![picture of Boston skyline from the Charles River](https://images.unsplash.com/photo-1560799260-b737af7dd0fc?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=2462&q=80)
_Photo by Prateek Pisat on [Unsplash](https://unsplash.com/photos/SFsKWydc7qY)_

# Mapping Boston's Food Scene
Prepared by Brandon Bellanti<br>
Last updated April 25, 2021

---

## Introduction/Business Problem

In my project, I will analyze and cluster food venues across Boston. I'll look at aspects like type of food, price, rating, hours, and location.

Some of the questions I'm interested in answering are:

* Are restaurants of a given food type distrubuted evenly across the city, or are they clustered together? If they're clustered, is there a correlation between the type of food and the demographics of that neighborhood?
* Which neighborhoods in Boston are the most similar in terms of food venues? Which neighborhoods are most different? 
* Does average food venue price correspond with income, either per-capita or by household?

The audience for this analysis is anyone interested in exploring Boston's food scene. One practical way it could be used is by someone who wants to open a restaurant in Boston: they would be able to see where similar restaurants – their competition – already exist or perhaps do not exist.

## Data

My data will come from two main sources, Boston's open data hub – [Analyze Boston](https://data.boston.gov/) – and the Foursquare API.

I'll use datasets available on Analyze Boston for [neighborhood](http://bostonopendata-boston.opendata.arcgis.com/datasets/3525b0ee6e6b427f9aab5d0a1d0a1a28_0.csv) and possibly [demographic data](https://data.boston.gov/dataset/neighborhood-demographics/resource/d8c23c6a-b868-4ba4-8a3b-b9615a21be07?view_id=875bddaa-ffb9-4eaa-bdb5-1b77b414182b) like income or mobility.

I'll use the Foursquare API for venue data. Once I have coordinate data for each Boston neighborhood (returned using the geopy library and the neighborhood data from Analyze Boston), I will call the `/venues` endpoint of the Foursquare API and search for nearby venues that have the categoryId `Food` (4d4b7105d754a06374d81259).

Searching by categoryId also includes all venues in child categories and their children, so I will group the venues by common parent categories to have fewer values overall. Since the results of the standard `/venues` endpoint does not contain parent category information, I'll build a separate reference dataframe with the hierarchy on the Foursquare API [Venue Categories](https://developer.foursquare.com/docs/build-with-foursquare/categories/) page. 

For each venue, I am interested in details such as price and rating, as well as opening hours and popular times. These details are accessed through a Premium endpoint - `/venues/VENUE_ID` – and I am limited to 500 Premium calls per day, so I will store the returned data to CSV files for each neighborhood if I hit my premium call limit. This way, if necessary, I can build a small cache of data over a couple of days and collect all the data I need.

I will use the K-Nearest-Neighbor algorithm to cluster the venues.